# Reading Notes Thrase.jl

In [5]:
using DifferentialEquations

In [3]:
# bdry_vec_mod!()
# F matrices are associated with imposing boundary conditions
# HfI_FT matrices are associated with computing traction

# G1 = -(Iz ⊗ ex0T) * Sx0 
# G2 = +(Iz ⊗ exNT) * SxN 
# G3 = -(ez0T ⊗ Ix) * Sz0
# G4 = +(ezNT ⊗ Ix) * SzN

# F1 = G1' - ((τ1 * H1) ⊗ ex0)
# F2 = G2' - ((τ2 * H2) ⊗ exN)
# F3 = G3' - (ez0 ⊗ (τ3 * H3))
# F4 = G4' - (ezN ⊗ (τ4 * H4))


# HfI_F1T = H1I * G1 - (τ1 ⊗ ex0')
# HfI_F2T = H2I * G2 - (τ2 ⊗ exN')
# HfI_F3T = H3I * G3 - (ez0' ⊗ τ3)
# HfI_F4T = H4I * G4 - (ezN' ⊗ τ4)

# g[:] .= 0

# # FACE 1 (Dirichlet):
# vf = bc_Dirichlet(1, 0, z)
# g[:] -= F[1] * vf

# # FACE 2 (Dirichlet):
# vf = bc_Dirichlet(2, Lx, z)
# g[:] -= F[2] * vf

# # FACE 3 (Neumann):
# gN = bc_Neumann(3, x, 0, 0, -1)
# vf = gN  ./ diag(τ[3])
# g[:] -= F[3] * vf

# # FACE 4 (Neumann):
# gN = bc_Neumann(4, x, Lz, 0, 1)
# vf = gN  ./ diag(τ[4])
# g[:] -= F[4] * vf 

In [ ]:
cb_fun = SavingCallback(
    (ψδ, t, i) -> write_to_file(pth, ψδ, t, i, z, flt_loc, flt_loc_indices,station_strings, station_indices, odeparam, "BP1_", 0.1 * year_seconds), 
    SavedValues(Float64, Float64)
)
